#Spectral Methods in Data Processing (Fall 2019) - Final Project
## Random SVD and Random ID Results Reconstruction- Elad Eatah

<a href="https://colab.research.google.com/github/RedCrow9564/SpectralMethodsProject-RandomSVD/blob/master/Spectral_Methods_Project_Random_SVD.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>
[![MIT License](https://img.shields.io/apm/l/atomic-design-ui.svg?)](https://github.com/tterb/atomic-design-ui/blob/master/LICENSEs)

Complete description of this project is avilable in [this repo](https://github.com/RedCrow9564/SpectralMethodsProject-RandomSVD.git).

## Getting Started
First run the nodes under "Infrastructure" one by one.
Second, run the first two nodes under "Main Components".

### Running Unit-Tests
You can then run the nodes under "UnitTests" one by one. The results of the last node are the results of these Unit-Tests.

### Performing an experiment
This is done by running the "main" node. The number of the experiment to perform can be set in the form. It can be any integer between 1 and 5. Its results are then saved to a local directory in the remote machine. Make sure this directory exists on the remote machine.


#Infrastructure

In [0]:
#@title Dependencies installations
!pip install pandas nptyping sacred
from IPython.display import clear_output
import numpy as np
import warnings
import psutil
from multiprocessing import Pool
import pandas as pd
import os
import pyximport
import cython
from sacred import Experiment
from time import perf_counter
import cpuinfo
#warnings.filterwarnings("error")  # Uncomment to see warnings as errors.
%load_ext Cython

# Defining the "sacred" experiment object.
ex = Experiment(name="Initializing project", interactive=True)

cpu_info = cpuinfo.get_cpu_info()
cpu_count: int = cpu_info['count']
clear_output()
print(cpu_info['brand'])
print(f'Total CPUs: {cpu_count}')
print("Installation is done!")


Intel(R) Xeon(R) CPU @ 2.20GHz
Total CPUs: 2
Installation is done!


In [0]:
#@title Common Utils
# -*- coding: utf-8 -*-
"""
utils.py - The common utilities functions and objects
=====================================================

This module contains all frequently-used methods and objects which can be shared among the entire project.
For example, data types name used for type-hinting, a basic enum class :class:`BaseEnum`, methods for measuring
run-time of a given function.
"""
from typing import List, Dict, Callable, Union, Iterator
from nptyping import Array
import inspect

pyximport.install(setup_args={"include_dirs": np.get_include()}, 
                  reload_support=True)

# Naming data types for type hinting.
Number = Union[int, float]
Scalar = Union[Number, Array[float, 1, 1], Array[int, 1, 1]]
RowVector = Union[List[Scalar], Array[float, 1, ...], Array[int, 1, ...], Scalar]
ColumnVector = Union[List[Scalar], Array[float, ..., 1], Array[int, ..., 1], Scalar]
Vector = Union[RowVector, ColumnVector]
Matrix = Union[List[Vector], Array[float], Array[int], Vector, Scalar]


class _MetaEnum(type):
    """
    A private meta-class which given any :class:`BaseEnum` object to be an iterable.
    This can be used for iterating all possible values of this enum. Should not be used explicitly.
    """
    def __iter__(self) -> Iterator:
        """
        This method gives any BaseEnum the ability of iterating over all the enum's values.

        Returns:
            An iterator for the collection of all the enum's values.

        """
        # noinspection PyUnresolvedReferences
        return self.enum_iter()

    def __contains__(self, item) -> bool:
        """
        This method give any BaseEnum the ability to test if a given item is a possible value for this enum class.

        Returns:
            A flag which indicates if 'item' is a possible value for this enum class.

        """
        # noinspection PyUnresolvedReferences
        return self.enum_contains(item)


class BaseEnum(metaclass=_MetaEnum):
    """
    A basic interface for all enum classes. Should be sub-classed in eny enum, i.e ``class ExperimentType(BaseEnum)``
    """

    @classmethod
    def enum_iter(cls) -> Iterator:
        """
        This method gives any BaseEnum the ability of iterating over all the enum's values.

        Returns:
            An iterator for the collection of all the enum's values.

        """
        return iter(cls.get_all_values())

    @classmethod
    def enum_contains(cls, item) -> bool:
        """
        This method give any BaseEnum the ability to test if a given item is a possible value for this enum class.

        Returns:
            A flag which indicates if 'item' is a possible value for this enum class.

        """
        return item in cls.get_all_values()

    @classmethod
    def get_all_values(cls) -> List:
        """
        A method which fetches all possible values of an enum. Used for iterating over an enum.

        Returns:
            A list of all possible enum's values.

        """
        all_attributes: List = inspect.getmembers(cls, lambda a: not inspect.ismethod(a))
        all_attributes = [value for name, value in all_attributes if not (name.startswith('__') or name.endswith('__'))]
        return all_attributes


def create_factory(possibilities_dict: Dict[str, Callable], are_methods: bool = False) -> Callable:
    """
    A generic method for creating factories for the entire project.

    Args:
        possibilities_dict(Dict[str, Callable]): The dictionary which maps object types (as strings!) and returns the
            relevant class constructors.
        are_methods(bool): A flag, true if the factory output are methods, rather than objects. Defaults to False

    Returns:
         The factory function for the given classes/methods mapping.

    """
    def factory_func(requested_object_type: str):  # Inner function!
        if requested_object_type not in possibilities_dict:
            raise ValueError("Object type {0} is NOT supported".format(requested_object_type))
        else:
            if are_methods:
                return possibilities_dict[requested_object_type]
            else:
                return possibilities_dict[requested_object_type]()

    return factory_func


def measure_time(method: Callable) -> Callable:
    """
    A method which receives a method and returns the same method, while including run-time measure
    output for the given method, in seconds.

    Args:
        method(Callable): A method whose run-time we are interested in measuring.

    Returns:
        A function which does exactly the same, with an additional run-time output value in seconds.

    """
    def timed(*args, **kw):
        ts = perf_counter()
        result = method(*args, **kw)
        te = perf_counter()
        duration_in_seconds: float = te - ts
        return result + (duration_in_seconds,)
    timed.__name__ = method.__name__ + " with time measure"
    return timed


def is_empty(collection: List) -> bool:
    return len(collection) == 0


class DataLog:
    """
    A class for log-management objects. See the following example for creating it: ``DataLog(["Column 1", "Column 2"])``
    """
    def __init__(self, log_fields: List):
        """
        This methods initializes an empty log.

        Args:
            log_fields(List) - A list of column names for this log.

        """
        self._data: Dict = dict()
        self._log_fields: List = log_fields
        for log_field in log_fields:
            self._data[log_field] = []

    def append(self, data_type: str, value: Scalar) -> None:
        """
        This methods appends given data to the given column inside the log.
        Example of usage:``log.append(DataFields.DataSize, 20)``

        Args:
            data_type(LogFields): The column name in which the input data in inserted to.
            value(Scalar): The value to insert to the log.

        """
        self._data[data_type].append(value)

    def append_dict(self, data_dict: Dict) -> None:
        """
        This methods takes the data from the input dictionary and inserts it to this log.

        Args:
            data_dict(Dict): The dictionary from which new data is taken and inserted to the log.

        """
        for log_field, data_value in data_dict.items():
            self.append(log_field, data_value)

    def save_log(self, log_file_name: str, results_folder_path: str) -> None:
        """
        This method saves the log to a file, with the input name, in the input folder path.

        Args:
            log_file_name(str): The name for this log file.
            results_folder_path(str): The path in which this log will be saved.

        """
        df = pd.DataFrame(self._data, columns=self._log_fields)
        df.to_csv(os.path.join(results_folder_path, log_file_name + ".csv"), sep=",", float_format="%.2E", index=False)
        ex.info["Experiment Log"] = self._data

print("Utils loaded successfully!")


Utils loaded successfully!


In [0]:
#@title Enums
# -*- coding: utf-8 -*-
"""
enums.py - All enums section
============================

This module contains all possible enums of this project. Most of them are used by the configuration section in
:mod:`main`. An example for using enum: ``ExperimentType.ExampleNo1``

"""

from typing import Iterator, List


class LogFields(BaseEnum):
    """
    The enum class of fields within experiments logs. Possible values:

    * ``LogFields.DataSize``

    * ``LogFields.ApproximationRank``

    * ``LogFields.Increment``

    * ``LogFields.NextSingularValue``

    * ``LogFields.RandomSVDDuration``

    * ``LogFields.RandomIDDuration``

    * ``LogFields.RandomSVDAccuracy``

    * ``LogFields.RandomIDAccuracy``
    """
    DataSize: str = "Data size"
    ApproximationRank: str = "k"
    Increment: str = "increment"
    NextSingularValue: str = "K+1 singular value"
    RandomSVDDuration: str = "Random SVD Duration in seconds"
    RandomIDDuration: str = "Random ID Duration in seconds"
    RandomSVDAccuracy: str = "Random SVD Accuracy"
    RandomIDAccuracy: str = "Random ID Accuracy"


class ExperimentType(BaseEnum):
    """
    The enum class of experiment types. Possible values:

    * ``ExperimentType.ExampleNo1``

    * ``ExperimentType.ExampleNo2``

    * ``ExperimentType.ExampleNo3``

    * ``ExperimentType.ExampleNo4``

    * ``ExperimentType.ExampleNo5``

    """
    ExampleNo1: str = "Example No. 1"
    ExampleNo2: str = "Example No. 2"
    ExampleNo3: str = "Example No. 3"
    ExampleNo4: str = "Example No. 4"
    ExampleNo5: str = "Example No. 5"

print("Enums loaded successfully!")


Enums loaded successfully!


#Main Components

In [0]:
#@title Data Loading for Experiments
# -*- coding: utf-8 -*-
""" The data management module

This module handles the fetching of the data from the local resources path, given in the configuration and arranging it
for our purposes of estimations. See the example for fetching the data for Example no. 1.

Example:
    get_data(ExperimentType.ExampleNo1) - Creating the data for Example no. 1 of the paper.

"""

from scipy.stats import ortho_group


def _random_orthonormal_cols(data_size: int, columns: int) -> Matrix:
    return ortho_group.rvs(data_size, size=1)[:, :columns]


def _get_first_3_examples_data(data_size: int, singular_values: RowVector) -> Matrix:
    """
    A method which creates a random matrix of size data_size x data_size with given singular values.

    Args:
        data_size(int): The input data size n.
        singular_values(RowVector): The singular values to be set for the matrix to create.

    Returns:
        A random size data_size x data_size Matrix awith the given singular values.

    """
    rank: int = len(singular_values)
    U: Matrix = _random_orthonormal_cols(data_size, rank)
    VT: Matrix = _random_orthonormal_cols(data_size, rank).T
    return U.dot(np.diag(singular_values).dot(VT))


def _get_example_4_data(data_size: int, singular_values: RowVector) -> Matrix:
    """
    A method which creates a data_size x data_size matrix whose singular values are the input values.

    Args:
        data_size(int): The input data size n.
        singular_values(RowVector): The singular values to be set for the matrix to create.

    Returns:
        A data_size x data_size Matrix with the given singular values.

    """
    U: Matrix = np.stack([np.ones(data_size),
                          np.tile([1, -1], data_size // 2),
                          np.tile([1, 1, -1, -1], data_size // 4),
                          np.tile([1, 1, 1, 1, -1, -1, -1, -1], data_size // 8)]).T / np.sqrt(data_size)
    VT: Matrix = np.stack([np.concatenate([np.ones(data_size - 1), [0]]) / np.sqrt(data_size - 1),
                           np.concatenate([np.zeros(data_size - 1), [1]]),
                           np.concatenate([np.tile([1, -1], (data_size - 2) // 2) / np.sqrt(data_size - 2), [0, 0]]),
                           np.concatenate([[1, 0, -1], np.zeros(data_size - 3)]) / np.sqrt(2)])
    return U.dot(np.diag(singular_values).dot(VT))


def _get_example_5_data(data_size: int, singular_values: RowVector) -> Matrix:
    """
    A method which creates a data_size x data_size matrix with singular values 1 and the other input singular values.

    Args:
        data_size(int): The input data size n.
        singular_value(RowVector): A 1x2 vector of singular values for the created matrix.

    Returns:
        A random size data_size x data_size Matrix with singular values 1 and the other input singular value.

    """
    data: Matrix = singular_values[1] * np.eye(data_size)
    data[0, :] += singular_values[0] * np.ones(data_size) / np.sqrt(data_size)
    return data


# A private dictionary used to create the method "get_data"
_data_type_to_function: Dict[str, Callable] = {
    ExperimentType.ExampleNo1: _get_first_3_examples_data,
    ExperimentType.ExampleNo2: _get_first_3_examples_data,
    ExperimentType.ExampleNo3: _get_first_3_examples_data,
    ExperimentType.ExampleNo4: _get_example_4_data,
    ExperimentType.ExampleNo5: _get_example_5_data
}

# The public method which fetches the data loading methods.
get_data: Callable = create_factory(_data_type_to_function, are_methods=True)

print("Data loading Methods loaded successfully!")


Data loading Methods loaded successfully!


In [0]:
#@title Randomized Decomposition Algorithms
%%cython
import numpy as np
cimport numpy as np
cimport cython
from numpy.linalg import svd
from scipy.linalg.interpolative import interp_decomp, reconstruct_interp_matrix

@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
@cython.cdivision(True)
@cython.initializedcheck(False)
def random_svd(np.ndarray[double, ndim=2] A, const int k, const int increment):
    cdef Py_ssize_t m = A.shape[0]
    cdef np.ndarray[double, ndim=1] sigma
    cdef np.ndarray[double, ndim=2] Q, VT, U

    _, _, Q = svd(np.random.randn(k + increment, m).dot(A), full_matrices=False,
                  compute_uv=True)
    Q = Q[:k, :].T
    U, sigma, VT = svd(A.dot(Q), full_matrices=False, compute_uv=True)
    VT = Q.dot(VT.T).T
    return U, sigma, VT


@cython.boundscheck(False)
@cython.wraparound(False)
@cython.nonecheck(False)
@cython.cdivision(True)
@cython.initializedcheck(False)
def random_id(np.ndarray[double, ndim=2] A, const int k, const int increment):
    cdef Py_ssize_t m = A.shape[0]
    cdef np.ndarray[int, ndim=1] idx
    cdef np.ndarray[double, ndim=2] B, P, proj

    idx, proj = interp_decomp(np.random.randn(k + increment, m).dot(A), k, 
                              rand=False)
    P = reconstruct_interp_matrix(idx, proj)
    B = A[:, idx[:k]]
    return B, P

print("Randommized Algorithms loaded successfully!")


Randommized Algorithms loaded successfully!


In [0]:
#@title Main
experiment_number_from_one_to_five = 1 #@param {type:"integer"}
#!/usr/bin/python
# -*- coding: utf-8 -*-
"""
main.py - The main module of the project
========================================

This module contains the config for the experiment in the "config" function.
Running this module invokes the :func:`main` function, which then performs the experiment and saves its results
to the configured results folder. Example for running an experiment: ``python -m main.py``

"""


def choose_singular_values(experiment_type: ExperimentType) -> RowVector:
    """

    This function sets the needed singular values, according to the given experiment_type

    Args:
        experiment_type(ExperimentType): The performed experiment. For example, ``ExperimentType.ExampleNo1``.

    Returns:
        A RowVector of the required singular values.

    """
    if experiment_type == ExperimentType.ExampleNo1:
        return np.concatenate([np.flip(np.geomspace(0.2e-15, 1, num=10)), 0.2e-15 * np.ones(10)])
    elif experiment_type == ExperimentType.ExampleNo2:
        return np.concatenate([np.flip(np.geomspace(1e-8, 1, num=10)), 1e-8 * np.ones(10)])
    elif experiment_type == ExperimentType.ExampleNo3:
        return np.concatenate([np.flip(np.geomspace(1e-9, 1, num=30)), 1e-9 * np.ones(30)])
    elif experiment_type == ExperimentType.ExampleNo4:
        return [1, 1, 1e-8, 1e-8]
    elif experiment_type == ExperimentType.ExampleNo5:
        return [1, 1e-17]
    return [-1]


def choose_increments(experiment_type: ExperimentType) -> List:
    """

    This function sets the needed increments, according to the given experiment_type

    Args:
        experiment_type(ExperimentType): The performed experiment. For example, ``ExperimentType.ExampleNo1``.

    Returns:
        A list of the required increments.

    """
    if experiment_type in [ExperimentType.ExampleNo1, ExperimentType.ExampleNo2,
                           ExperimentType.ExampleNo4, ExperimentType.ExampleNo5]:
        return [0]
    elif experiment_type == ExperimentType.ExampleNo3:
        return [0] + np.round(np.geomspace(2, 16, 4)).astype(int).tolist()
    return [-1]


def choose_approximation_ranks(experiment_type: ExperimentType) -> List:
    """

    This function sets the needed approximation ranks, according to the given experiment_type

    Args:
        experiment_type(ExperimentType): The performed experiment. For example, ``ExperimentType.ExampleNo1``.

    Returns:
        A list of the required approximation ranks.

    """
    if experiment_type in [ExperimentType.ExampleNo1, ExperimentType.ExampleNo2, ExperimentType.ExampleNo5]:
        return [10]
    elif experiment_type == ExperimentType.ExampleNo3:
        return [30]
    elif experiment_type == ExperimentType.ExampleNo4:
        return [2]
    return [-1]


def choose_data_sizes(experiment_type: ExperimentType) -> List:
    """

    This function sets the needed data sizes, according to the given experiment_type

    Args:
        experiment_type(ExperimentType): The performed experiment. For example, ``ExperimentType.ExampleNo1``.

    Returns:
        A list of the required data sizes.

    """
    if experiment_type in [ExperimentType.ExampleNo1, ExperimentType.ExampleNo2, ExperimentType.ExampleNo5]:
        return np.geomspace(1e+2, 1e+4, 3, dtype=int).tolist()
    elif experiment_type == ExperimentType.ExampleNo3:
        return [1e+5]
    elif experiment_type == ExperimentType.ExampleNo4:
        return (4 * np.geomspace(1e+2, 1e+4, 3, dtype=int)).tolist()
    return [-1]


@ex.config
def config():
    """ Config section

    This function contains all possible configuration for all experiments. Full details on each configuration values
    can be found in :mod:`enums.py`.
    """

    experiment_type: str = f'Example No. {experiment_number_from_one_to_five}'
    singular_values: RowVector = choose_singular_values(experiment_type)
    used_data_factory: Callable = get_data(experiment_type)
    data_sizes: List = choose_data_sizes(experiment_type)
    approximation_ranks: List = choose_approximation_ranks(experiment_type)
    increments: List = choose_increments(experiment_type)
    results_path: str = r'Results/'


@ex.main
def main(data_sizes: List, approximation_ranks: List, increments: List, singular_values: RowVector,
         used_data_factory: Callable, results_path: str, experiment_type: str) -> DataLog:
    """ The main function of this project

    This functions performs the desired experiment according to the given configuration.
    The function runs the random_svd and random_id for every combination of data_size, approximation rank and increment
    given in the config and saves all the results to a csv file in the results folder (given in the configuration).
    """
    results_log = DataLog(LogFields)  # Initializing an empty results log.
    random_svd_with_run_time: Callable = measure_time(random_svd)
    random_id_with_run_time: Callable = measure_time(random_id)

    for data_size in data_sizes:
        data_matrix: Matrix = used_data_factory(data_size, singular_values)

        for approximation_rank in approximation_ranks:
            next_singular_value: Scalar = singular_values[approximation_rank + 1] if \
                approximation_rank < len(singular_values) else singular_values[-1]

            for increment in increments:
                # Executing all the tested methods.
                U, sigma, VT, svd_duration = random_svd_with_run_time(data_matrix, approximation_rank, increment)
                random_svd_accuracy: Scalar = np.linalg.norm(data_matrix - U.dot(np.diag(sigma).dot(VT)))
                B, P, id_duration = random_id_with_run_time(data_matrix, approximation_rank, increment)
                random_id_accuracy: Scalar = np.linalg.norm(data_matrix - B.dot(P))

                # Appending all the experiment results to the log.
                results_log.append(LogFields.DataSize, data_size)
                results_log.append(LogFields.ApproximationRank, approximation_rank)
                results_log.append(LogFields.Increment, increment + approximation_rank)
                results_log.append(LogFields.NextSingularValue, next_singular_value)
                results_log.append(LogFields.RandomSVDAccuracy, random_svd_accuracy)
                results_log.append(LogFields.RandomIDAccuracy, random_id_accuracy)
                results_log.append(LogFields.RandomSVDDuration, svd_duration)
                results_log.append(LogFields.RandomIDDuration, id_duration)

    results_log.save_log(experiment_type + " results", results_folder_path=results_path)
    print(f'{experiment_type} was performed and results were saved!')
    return results_log

pd.DataFrame(ex.run().result._data)


WARNING - Initializing project - No observers have been added to this run
INFO - Initializing project - Running command 'main'
INFO - Initializing project - Started
INFO - Initializing project - Result: <__main__.DataLog object at 0x7ff85c8735c0>
INFO - Initializing project - Completed after 2:24:45


Example No. 1 was performed and results were saved!


,k,Data size,increment,K+1 singular value,Random ID Accuracy,Random ID Duration in seconds,Random SVD Accuracy,Random SVD Duration in seconds
0,10,100,10,2.000000e-16,2.864991e-15,0.000479,1.848750e-15,0.001168
1,10,1000,10,2.000000e-16,2.943775e-15,0.003867,3.510649e-15,0.006946
2,10,10000,10,2.000000e-16,2.917524e-15,0.296181,2.330714e-15,0.462816


#Unit-Tests

In [0]:
#@title Test data creation methods
# -*- coding: utf-8 -*-
"""
test_data_creation.py - tests for data creation methods
=======================================================

This module contains the tests for the data creation in all the examples.

"""
import unittest
from scipy.linalg import svdvals


class TestDataCreation(unittest.TestCase):
    """
    A class which contains tests for the validity of the created data in all the examples
    """
    def test_example_no_1_data(self):
        """
        Test data creation for Example no. 1

        This test validates the data created is ``data_size x data_size``, has rank 20
        and posses the expected singular values.

        """
        experiment_type: str = ExperimentType.ExampleNo1
        data_size: int = 70
        singular_values: RowVector = choose_singular_values(experiment_type)
        rank: int = len(singular_values)
        data: Matrix = get_data(experiment_type)(data_size, singular_values)
        calculated_singular_values: RowVector = svdvals(data, check_finite=False)[:rank]
        self.assertTrue(np.allclose(data.shape, (data_size, data_size)))  # Validate data shape.
        self.assertEqual(np.linalg.matrix_rank(data, tol=1.8e-16), rank)  # Validate data rank.
        self.assertTrue(np.allclose(singular_values, calculated_singular_values))  # Validate singular values.

    def test_example_no_2_data(self):
        """
        Test data creation for Example no. 2

        This test validates the data created is ``data_size x data_size``, has rank 20
        and posses the expected singular values.

        """
        experiment_type: str = ExperimentType.ExampleNo2
        data_size: int = 70
        singular_values: RowVector = choose_singular_values(experiment_type)
        rank: int = len(singular_values)
        data: Matrix = get_data(experiment_type)(data_size, singular_values)
        calculated_singular_values: RowVector = svdvals(data, check_finite=False)[:rank]
        self.assertTrue(np.allclose(data.shape, (data_size, data_size)))  # Validate data shape.
        self.assertEqual(np.linalg.matrix_rank(data, tol=singular_values[rank - 1] / 2), rank)  # Validate data rank.
        self.assertTrue(np.allclose(singular_values, calculated_singular_values))  # Validate singular values.

    def test_example_no_3_data(self):
        """
        Test data creation for Example no. 3

        This test validates the data created is ``data_size x data_size``, has rank 60
        and posses the expected singular values.

        """
        experiment_type: str = ExperimentType.ExampleNo3
        data_size: int = 70
        singular_values: RowVector = choose_singular_values(experiment_type)
        rank: int = len(singular_values)
        data: Matrix = get_data(experiment_type)(data_size, singular_values)
        calculated_singular_values: RowVector = svdvals(data, check_finite=False)[:rank]
        self.assertTrue(np.allclose(data.shape, (data_size, data_size)))  # Validate data shape.
        self.assertEqual(np.linalg.matrix_rank(data, tol=singular_values[rank - 1] / 2), rank)  # Validate data rank.
        self.assertTrue(np.allclose(singular_values, calculated_singular_values))  # Validate singular values.

    def test_example_no_4_data(self):
        """
        Test data creation for Example no. 4

        This test validates the data created is ``data_size x data_size`` and posses the expected singular values.

        """
        experiment_type: str = ExperimentType.ExampleNo4
        data_size: int = 80
        singular_values: RowVector = choose_singular_values(experiment_type)
        known_singular_values_num: int = len(singular_values)
        data: Matrix = get_data(experiment_type)(data_size, singular_values)
        calculated_singular_values: RowVector = svdvals(data, check_finite=False)[:known_singular_values_num]
        self.assertTrue(np.allclose(data.shape, (data_size, data_size)))  # Validate data shape.
        self.assertTrue(np.allclose(singular_values, calculated_singular_values))  # Validate singular values.

    def test_example_no_5_data(self):
        """
        Test data creation for Example no. 5

        This test validates the data created is ``data_size x data_size`` and posses the expected singular value
        :math:`10^{-17}` as the second largest singular value with multiplicity of at least ``data_size - 2``.

        """
        experiment_type: str = ExperimentType.ExampleNo5
        data_size: int = 70
        singular_values: RowVector = choose_singular_values(experiment_type)
        known_singular_values_num: int = data_size - 2
        data: Matrix = get_data(experiment_type)(data_size, singular_values)
        calculated_singular_values: RowVector = svdvals(data, check_finite=False)[1:known_singular_values_num + 1]
        known_singular_values: RowVector = singular_values[1] * np.ones(known_singular_values_num)
        self.assertTrue(np.allclose(data.shape, (data_size, data_size)))  # Validate data shape.
        self.assertTrue(np.allclose(known_singular_values, calculated_singular_values))  # Validate singular values.

print("Loaded Unit-Tests for data creation successfully!")


Loaded Unit-Tests for data creation successfully!


In [0]:
#@title Test Randomized Agorithms
# -*- coding: utf-8 -*-
"""
test_random_id.py - tests for Randomized Interpolative Decomposition
====================================================================

This module contains the tests for the implementation of randomized ID algorithm.

"""
from numpy.random import randn as _randn

pyximport.install(setup_args={"include_dirs": np.get_include()}, reload_support=True)


class TestRandomID(unittest.TestCase):
    """
    A class which contains tests for the validity of the random_id algorithm implementation.
    """
    def setUp(self):
        """
        This method sets the variables for the following tests.
        """
        self._m = 100
        self._n = 30
        self._k = 5
        self._increment = 20
        self._A = _randn(self._m, self._n)
        self._B, self._P = random_id(self._A, self._k, self._increment)
        self._approximation = self._B.dot(self._P)

    def test_matrices_shapes(self):
        """
        This methods tests the shapes of the matrices :math:`B` and :math:`P` in the decomposition.
        """
        self.assertTrue(self._B.shape, (self._m, self._k))
        self.assertTrue(self._P.shape, (self._k, self._n))

    def test_interpolative_decomposition(self):
        """
        This methods tests if the decomposition satisfies the properties of interpolative-decomposition.
        """
        self.assertTrue(np.all(self._P <= 2))  # Validate entries of P are between -1 and 2.
        self.assertTrue(np.all(self._P >= -2))
        # Validate P's norm is bound by the theoretical bound
        self.assertLessEqual(np.linalg.norm(self._P), np.sqrt(self._k * (self._n - self._k) + 1))
        self.assertGreaterEqual(svdvals(self._P)[-1], 1)  # Validate the least singular value of P is at least 1.

        for unit_vector in np.eye(self._k):  # Validate P has kxk identity matrix as a sub-matrix.
            self.assertIn(unit_vector, self._P.T)

        for col in self._B.T:  # Validate every column of B is also a column of A.
            self.assertIn(col, self._A.T)

    def test_approximation_estimate(self):
        """
        This methods tests if the random ID satisfies the theoretical bound. There is a probability
        of less then :math:`10^{-17}` this bound won't be satisfied...
        """
        real_sigmas = np.linalg.svd(self._A, full_matrices=False, compute_uv=False)
        estimate_error = np.linalg.norm(self._A - self._approximation)
        expected_bound = 10 * np.sqrt(self._n * (self._k + self._increment) * self._m * self._k)
        expected_bound *= real_sigmas[self._k]
        self.assertLessEqual(estimate_error, expected_bound)

print("Loaded Unit-Tests for Random ID successfully!")


Loaded Unit-Tests for Random ID successfully!


In [0]:
#@title Test Randomized SVD
# -*- coding: utf-8 -*-
"""
test_random_svd.py - tests for Randomized Singular-Value Decomposition
======================================================================

This module contains the tests for the implementation of randomized SVD algorithm.

"""

pyximport.install(setup_args={"include_dirs": np.get_include()}, reload_support=True)


class TestRandomSVD(unittest.TestCase):
    """
    A class which contains tests for the validity of the random_svd algorithm implementation.
    """
    def setUp(self):
        """
        This method sets the variables for the following tests.
        """
        self._m = 100
        self._n = 30
        self._k = 5
        self._increment = 20
        self._A = _randn(self._m, self._n)
        self._U, self._sigma, self._VT = random_svd(self._A, self._k, self._increment)
        self._approximation = self._U.dot(np.diag(self._sigma).dot(self._VT))

    def test_matrices_shapes(self):
        """
        This methods tests the shapes of the matrices :math:`U` and :math:`V` in the decomposition.
        """
        self.assertTrue(self._U.shape, (self._m, self._k))
        self.assertTrue(self._VT.shape, (self._k, self._n))

    def test_matrices_svd_decomposition(self):
        """
        This methods tests if the output decomposition satisfies the properties of SVD decomposition.
        """
        self.assertTrue(np.allclose(self._U.T.dot(self._U), np.eye(self._k)))
        self.assertTrue(np.allclose(self._VT.dot(self._VT.T), np.eye(self._k)))
        self.assertTrue(np.all(self._sigma > 0))

    def test_decomposition_rank(self):
        """
        This methods tests if the number of positive singular values is equal to the approximation rank.
        """
        self.assertEqual(len(self._sigma), self._k)

    def test_approximation_estimate(self):
        """
        This methods tests if the random SVD satisfies the theoretical bound. There is a probability
        of less then :math:`10^{-17}` this bound won't be satisfied...
        """
        real_sigmas = np.linalg.svd(self._A, full_matrices=False, compute_uv=False)
        estimate_error = np.linalg.norm(self._A - self._approximation)
        expected_bound = 10 * np.sqrt(self._n * (self._k + self._increment))
        expected_bound *= real_sigmas[self._k]
        self.assertLessEqual(estimate_error, expected_bound)

print("Loaded Unit-Tests for Random SVD successfully!")


Loaded Unit-Tests for Random SVD successfully!


In [0]:
#@title Running all these Unit-Tests
unittest.main(argv=[''], verbosity=2, exit=False)


test_example_no_1_data (__main__.TestDataCreation) ... ok
test_example_no_2_data (__main__.TestDataCreation) ... ok
test_example_no_3_data (__main__.TestDataCreation) ... ok
test_example_no_4_data (__main__.TestDataCreation) ... ok
test_example_no_5_data (__main__.TestDataCreation) ... ok
test_approximation_estimate (__main__.TestRandomID) ... ok
test_interpolative_decomposition (__main__.TestRandomID) ... ok
test_matrices_shapes (__main__.TestRandomID) ... ok
test_approximation_estimate (__main__.TestRandomSVD) ... ok
test_decomposition_rank (__main__.TestRandomSVD) ... ok
test_matrices_shapes (__main__.TestRandomSVD) ... ok
test_matrices_svd_decomposition (__main__.TestRandomSVD) ... ok

----------------------------------------------------------------------
Ran 12 tests in 0.083s

OK
